In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
import ctapipe
from lstchain.io import replace_config, standard_config
import h5py
from astropy.io import fits
import json
import os
filename = '/Users/vdk/simtel_corsika_theta_30.390_az_93.640_run1.simtel.gz'
source = EventSource(filename)
event_iterator = iter(source)

## Observations

In [ ]:
event_waveforms = {}
muon_waveforms_obs = []
foldername = '/Users/vdk/RealLST/waveforms1subrun/'
for filename in os.listdir(foldername):
    muon_waveforms_obs.append(np.load(foldername + filename, allow_pickle=True))


flag = True
counter = 0
counter = 0
proton_waveforms_obs = []

foldername = '/Users/vdk/RealLST/waveforms1subrun_lot/'
for filename2 in os.listdir(foldername):
  proton_waveforms_obs.append(np.load(foldername + filename2, allow_pickle=True))
  if counter == 1000:
    break



min_time = 6
max_time = 14

arr_max_muons = []
arg_max_muons = []

arg_max_muons_mask = []


variance_muon = []


muon_pixel_number = []

for waveform in muon_waveforms_obs:
   tmp_var_muon = []
   tmp_pixel = 0
   pixel_mask = np.array(np.max(waveform, axis = 1) > 2.5, dtype = bool)
   for pixel in waveform[pixel_mask]:
      arg_max_muons_mask.append(np.argmax(pixel))
      tmp_pixel += 1
    
   for pixel in waveform[:, min_time:max_time]:
      tmp_var_muon.append(np.var(pixel))
   
   variance_muon.append(tmp_var_muon)
   muon_pixel_number.append(tmp_pixel)


arg_max_protons = []
arg_max_protons_mask = []
variance_proton = []
proton_pixel_number = []
stopper = 0

for waveform in proton_waveforms_obs:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2.5, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton.append(tmp_var_proton)
   proton_pixel_number.append(tmp_pixel)
   stopper += 1
   if stopper == 10000:
      break


## Simulations

In [ ]:
event_waveforms = {}
muon_waveforms = []
for filename in os.listdir('/Users/vdk/UnrealLST/waveforms/'):
    event_waveforms[filename[10:16]] = np.load('/Users/vdk/UnrealLST/waveforms/' + filename)
    #muon_waveforms.append( np.load('/Users/vdk/RealLST/waveforms/' + filename, allow_pickle=True))


proton_waveforms = []
flag = True
counter = 0


counter = 0
proton_waveforms = []
for filename2 in os.listdir('/Users/vdk/UnrealLST/waveformsProton/'):
  proton_waveforms.append(np.load('/Users/vdk/UnrealLST/waveformsProton/' + filename2, allow_pickle=True))
  if counter == 1000:
    break



min_time = 9
max_time = 14

arg_max_muons_mask_sim = []

variance_muon_sim = []


muon_pixel_number_sim = []

for key in event_waveforms:
   tmp_var_muon = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(event_waveforms[key], axis = 1) > 2, dtype = bool)
      for pixel in event_waveforms[key][pixel_mask]:
        arg_max_muons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in event_waveforms[key][:, min_time:max_time]:
        tmp_var_muon.append(np.var(pixel))
   
   variance_muon_sim.append(tmp_var_muon)
   muon_pixel_number_sim.append(tmp_pixel)



arg_max_protons_mask_sim = []

variance_proton_sim = []

proton_pixel_number_sim = []

for waveform in proton_waveforms:
   tmp_var_proton = []
   tmp_pixel = 0
   if True:
      pixel_mask = np.array(np.max(waveform, axis = 1) > 2, dtype = bool)
      for pixel in waveform[pixel_mask]:
        arg_max_protons_mask_sim.append(np.argmax(pixel))
        tmp_pixel += 1
      
      for pixel in waveform[:, min_time:max_time]:
        tmp_var_proton.append(np.var(pixel))

   variance_proton_sim.append(tmp_var_proton)
   proton_pixel_number_sim.append(tmp_pixel)


## Variance

In [ ]:
hist_density = False
#upper_y = 20
upper_y = 100

var_hist_m_short = [np.mean(np.array(item)) for item in variance_muon] # only for considered time span (7:12)
var_hist_p_short = [np.mean(np.array(item)) for item in variance_proton]
var_hist_m_short_sim = [np.mean(np.array(item)) for item in variance_muon_sim] # only for considered time span (7:12)
var_hist_p_short_sim = [np.mean(np.array(item)) for item in variance_proton_sim]
bin_number = 30
plt.figure(figsize = (9,6))
z,x,c = plt.hist(var_hist_p_short, alpha = 0.35, density = hist_density,color = '#1f77b4', bins = bin_number, range =(0,1), label='not muons in data')
plt.hist(var_hist_m_short, alpha = 0.35, density = hist_density, color = 'orange', bins = bin_number, range=(0,1), label='muons in data')

#plt.hist(var_hist_p_short_sim, alpha = 1, density = True, color = 'red', bins = bin_number, range =(0,1), label='not muons in simulation', histtype='step')
#plt.hist(var_hist_m_short_sim, alpha = 1, density = True, bins = bin_number, color = 'green', range=(0,1), label='muons in simulation', histtype='step')

plt.grid(alpha = 0.5)
plt.legend()
plt.ylim(0,upper_y)
plt.xlim(0,0.6)
plt.xlabel('Mean of variance per pixel for every event')
#plt.ylabel('Probability density')
plt.ylabel('Count')
#plt.title(f'Run{15085} Distribution of mean for variance per pixel for timebins [{min_time}:{max_time}] - data; [9:14] - sim')
plt.title(f'Run{15085} Distribution of mean for variance per pixel for timebins [{min_time}:{max_time}]')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

In [ ]:
print(sorted(var_hist_m_short, reverse=True))

## Time of waveform max

In [ ]:
hist_density = True
plt.figure(figsize = (9,6))
bin_number1 = 35
bin_number2 = 40
# Time distribution
k, z, l = plt.hist(arg_max_protons_mask, bins = bin_number1, color = '#1f77b4', density = hist_density, alpha = 0.35, label = 'not muons in data')
muons_val, muons_bin, l = plt.hist(arg_max_muons_mask, bins = bin_number1, density = hist_density,color = 'orange', alpha = 0.35, label = 'muons in data')

#k, z, l = plt.hist(arg_max_protons_mask_sim, bins = bin_number2, density = True, color = 'red', alpha = 1, label = 'not muons in simulation', histtype = 'step')
#k, z, l = plt.hist(arg_max_muons_mask_sim, bins = bin_number2, density = True, color = 'green', alpha = 1, label = 'muons in simulation', histtype = 'step')




plt.legend()
plt.xlabel("Time of max R1 amplitude")
plt.ylabel("Probability density")
plt.grid(alpha = 0.5)
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_timesample_R1cut3.png")


## Number of Pixels

In [ ]:
hist_density = False
#upper_y = 0.1

protons_outlier = [250 for i in proton_pixel_number if i >= 250]
protons_outlier_sim = [250 for i in proton_pixel_number_sim if i >= 250]

final_proton_number = proton_pixel_number + protons_outlier
final_proton_number_sim = proton_pixel_number_sim + protons_outlier_sim

bin_number = 25
plt.figure(figsize = (9,6))
z,x,c = plt.hist(final_proton_number, alpha = 0.35, density = hist_density,color = '#1f77b4', bins = bin_number, range =(0,250), label='not muons in data')
plt.hist(muon_pixel_number, alpha = 0.35, density = hist_density, color = 'orange', bins = bin_number, range=(0,250), label='muons in data')

#plt.hist(final_proton_number_sim, alpha = 1, density = True, color = 'red', bins = bin_number, range =(0,250), label='not muons in simulation', histtype='step')
#plt.hist(muon_pixel_number_sim, alpha = 1, density = True, bins = bin_number, color = 'green', range=(0,250), label='muons in simulation', histtype='step')

plt.grid(alpha = 0.5)
plt.legend()
plt.ylim(0,upper_y)
plt.xlabel('Number of pixels on the image with R1 amplitude > 2.5')
#plt.ylabel('Probability density')
plt.ylabel('Count')
plt.title('Distribution of pixel number for simulation and real data (overflow bin 250)')
#plt.savefig("/Users/vdk/RealLST/images/Run14948_1_variance_R1cut2_7-11timebin.png")

In [ ]:
print(sorted(muon_pixel_number))

## Muon Images

In [ ]:
for i,waveform in enumerate(muon_waveforms_obs):
    plt.figure(figsize = (18,12))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'{i}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.highlight_pixels(pixel_mask)
    disp.image = waveform.sum(axis = 1)
    disp.cmap = plt.cm.Reds
    disp.add_colorbar()
    disp.set_limits_percent(95)
    #disp.set_limits_minmax(0,20)
    plt.show()